<a href="https://colab.research.google.com/github/ymubarka/finetuning_methods/blob/main/LP_FT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Inspired by this paper "[Fine-Tuning can Distort Pretrained Features and Underperform Out-of-Distribution](https://arxiv.org/pdf/2202.10054.pdf)", I will be implementing one of their experiments where they train a linear probe and finetune the whole network. Check out their paper and [codebase](https://github.com/AnanyaKumar/transfer_learning) if you're interested!

## Steps:
0. Get data (CIFAR10) and pre-trained model (MoCo v2)
1. Get the features from the pre-trained model
2. Train the Linear Probe (LP) on the CIFAR10 features
  * Using a torch Linear layer unlike the authors
3. Replace the pre-trained model's fully connected layer with the LP
4. Finetune for 20 epochs

## Getting the data and models

In [ ]:
# Args
batch_size = 256
num_workers = 8

lr = 3e-3
epochs = 20
T_max = epochs
momentum = 0.9

In [ ]:
import numpy as np
import torch
import torchvision.models as models
from torchvision.datasets import CIFAR10
from torchvision import transforms
from tqdm.notebook import tqdm

import os


def load_moco(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location="cpu")
    model = models.__dict__[checkpoint['arch']]()

    state_dict = checkpoint['state_dict']
    for k in list(state_dict.keys()):
        # retain only encoder_q up to before the embedding layer
        if k.startswith('module.encoder_q') and not k.startswith('module.encoder_q.fc'):
            # remove prefix
            state_dict[k[len("module.encoder_q."):]] = state_dict[k]
        # delete renamed or unused k
        del state_dict[k]
    msg = model.load_state_dict(state_dict, strict=False)
    assert set(msg.missing_keys) == {"fc.weight", "fc.bias"}
    return model

def train(model, data_loader, optimizer, criterion, epoch, use_cuda):
    model.train()
    total_loss, train_bar = 0.0, tqdm(data_loader)
    correct = 0
    total = 0
    for data, labels in train_bar:
        if use_cuda:
            data, labels = data.cuda(non_blocking=True), labels.cuda(non_blocking=True)
        
        optimizer.zero_grad()
        logits = model(data)
        _, predicted = torch.max(logits.data, axis=1)

        loss = criterion(logits, labels)
        correct += sum((predicted == labels).tolist())

        loss.backward()
        optimizer.step()

        total += labels.size(0)
        total_loss += loss.item() 
        acc = correct / total * 100
        train_bar.set_description(f"Train Epoch: [{epoch+1}/20], Avg. Loss: {total_loss / total:.4f}, Avg. Acc: {acc:.2f}")

    return total_loss / total

def test(model, data_loader, epoch, use_cuda):
    model.eval()
    total_acc, total_num, test_bar = 0.0, 0, tqdm(data_loader)
    with torch.no_grad():
      for data, labels in test_bar:
          if use_cuda:
              data = data.cuda(non_blocking=True)

          logits = model(data)
          _, test_preds = torch.max(logits.data, axis=1)

          total_num += data.size(0)
          total_acc += (test_preds.cpu() == labels).float().sum().item()
          acc = total_acc / total_num * 100

          test_bar.set_description(f"Test Epoch: [{epoch+1}/20], Avg. Acc: {acc:.2f}")

use_cuda = torch.cuda.is_available()

# Data Prepping
default_transformations = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.228, 0.224, 0.225])
    ])

cifar10_train = CIFAR10(root='drive/MyDrive/finetuning_project/data/', train=True, transform=default_transformations, download=True)
cifar10_test = CIFAR10(root='drive/MyDrive/finetuning_project/data/', train=False, transform=default_transformations, download=True)

train_loader = torch.utils.data.DataLoader(cifar10_train,
                                          batch_size=batch_size,
                                          num_workers=num_workers,
                                          shuffle=True,
                                          pin_memory=True)
test_loader = torch.utils.data.DataLoader(cifar10_test,
                                          batch_size=batch_size,
                                          num_workers=num_workers,
                                          shuffle=False,
                                          pin_memory=True)


# I got the pretrained model from the original moco repo from facebookresearch
# https://github.com/facebookresearch/moco
model = load_moco("drive/MyDrive/finetuning_project/models/moco_v2_800ep_pretrain.pth.tar")

Files already downloaded and verified
Files already downloaded and verified


## Extracting Features



In [ ]:
model.fc = torch.nn.Identity()
if use_cuda:
    model = model.cuda()

model.eval()
train_features = []
train_labels = []

with torch.no_grad():
    for data, labels in tqdm(train_loader):
        if use_cuda:
            data, labels = data.cuda(), labels.cuda() 
        features = model(data)
        train_features.append(features.detach().cpu().numpy())
        train_labels.append(labels.detach().cpu().numpy())

train_features = np.squeeze(np.concatenate(train_features))
train_labels = np.concatenate(train_labels)


train_features_dataset = torch.utils.data.TensorDataset(torch.Tensor(train_features), torch.Tensor(train_labels).long())
train_features_dataloader = torch.utils.data.DataLoader(train_features_dataset, batch_size=batch_size, shuffle=True) 

  0%|          | 0/196 [00:00<?, ?it/s]

# Training the LP

In [ ]:
lp_epochs = 200
lp_T_max = lp_epochs
lp_lr = 10  # [10, 1, 1e-1]
lp_momentum = 0.9

fc = torch.nn.Linear(2048, 10)
if use_cuda:
    fc = fc.cuda()

lp_criterion = torch.nn.CrossEntropyLoss(reduction='mean')
lp_optimizer = torch.optim.SGD(
    fc.parameters(), 
    lr=lp_lr, 
    momentum=lp_momentum, 
    weight_decay=0.0005)
lp_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    lp_optimizer, 
    lp_T_max)


for epoch in range(lp_epochs):
    train(fc, train_features_dataloader, lp_optimizer, lp_criterion, epoch, use_cuda)

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

...

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

## Replacing FC layer and finetuning

In [ ]:
exp_type = "LP-FT"
model.fc = fc  # Changing it from Identity to the Linear layer I just trained

results_dir = f"drive/MyDrive/finetuning_project/trained_models/{exp_type}"
if not os.path.exists(results_dir):
    os.mkdir(results_dir)

if use_cuda:
    model = model.cuda()

criterion = torch.nn.CrossEntropyLoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max)

for epoch in range(epochs):
    train(model, train_loader, optimizer, criterion, epoch, use_cuda)
    scheduler.step()
    torch.save({'epoch': epoch, 
                'state_dict': model.state_dict(), 
                'optimizer' : optimizer.state_dict(),}, results_dir + '/model_last.pth')
    test(model, test_loader, epoch, use_cuda)

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

...

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/196 [00:00<?, ?it/s]

  0%|          | 0/40 [00:00<?, ?it/s]

In [ ]:
from google.colab import runtime
runtime.unassign()